In [31]:
import requests
import pickle
import socket
import select
import threading

class BaseProtocol():
    def __init__(self, socket):
        self.host, self.port = socket.getsockname()
        self.sock = socket
        self.wfile = socket.makefile("wb")
        self.rfile = socket.makefile("rb")

    def _send(self, message, payload=None):
        obj = (message, payload)
        err = None
        for i in range(2):
            try:
                pickle.dump(obj, self.wfile)
                self.wfile.flush()
            except OSError as e:
                err = e
                self._reinit()
            else:
                return
        
        print("Error occurred sending response to %s"%repr(self.sock.getpeername()))
        raise

    def _recv(self):
        r, w, l = select.select([self.sock], [], [], 30)
        if r:
            msg, data = pickle.load(self.rfile)
        else:
            msg, data = "TIMEOUT", None
        return msg, data
    
    def _reinit(self):
        self.close()
        sock = socket.socket()
        sock.connect((self.host, self.port))
        super().__init__(sock)

    def close(self):
        for f in self.wfile, self.rfile:
            if f:
                try:
                    f.flush()
                    f.close()
                except OSError:
                    pass
        self.wfile = None
        self.rfile = None
        try:
            self.sock.shutdown(socket.SHUT_RDWR)
            self.sock.close()
        except OSError:
            pass


class ServerProtocol(BaseProtocol):
    def __init__(self, socket, sess, cache, lock):
        super().__init__(socket)
        self.sess = sess
        self.cache = cache
        self.lock = lock

    def get(self, url, kw):
        kws = " ".join('%r=%r'%i for i in kw.items())
        key = (url, kws)
        with self.lock:
            if key in self.cache:
                print("returning cached value for '%s'"%(repr(url)))
                return "SUCCESS", self.cache[key]
        print('requesting new value for "%s"'%(repr(key)))
        timeout = kw.pop('timeout', 40)
        try:
            rsp = self.sess.get(url, timeout=timeout, **kw)
        except Exception as e:
            return "ERR_RAISE_EXC", e
        rsp.content  # trigger actual read
        with self.lock:
            self.cache[key] = rsp
        return "SUCCESS", rsp
    
    def clear_cache(self):
        with self.lock:
            self.cache.clear()
        print("cleared internal cache")
        return "SUCCESS", "CLEAR_CACHE"

    def process(self):
        msg, data = self._recv()
        if msg == "GET_URL":
            url, kw = data
            msg, rsp = self.get(url, kw)
        elif msg == "CLEAR_CACHE":
            msg, rsp = self.clear_cache()
        elif msg == "TIMEOUT":
            return False
        else:
            msg, rsp = "UNKN_CMD", msg
        self._send(msg, rsp)
        return True
    
        
class ClientProtocol(BaseProtocol):

    def __init__(self, host, port):
        self.host = host
        self.port = port
        sock = socket.socket()
        sock.connect((self.host, self.port))
        super().__init__(sock)

    def clear_cache(self):
        self._send("CLEAR_CACHE", None)
        msg, rsp = self._recv()
        if msg != "SUCCESS":
            raise ValueError(rsp)

    def get(self, url, **kw):
        self._send("GET_URL", (url, kw))
        rsp, data = self._recv()
        if rsp == "SUCCESS":
            return data
        elif rsp == "ERR_RAISE_EXC":
            print("GOT ERROR")
            raise data
        else:
            raise ValueError("Unknown result returned: (%r, %r)"%(rsp, data))


class HandleIt(threading.Thread):
    def __init__(self, sock, cache, sess, lock):
        super().__init__(daemon=True)
        self.sock = sock
        self.cache = cache
        self.sess = sess
        self.lock = lock
        self.start()

    def run(self):
        proto = ServerProtocol(self.sock, self.sess, self.cache, self.lock)
        run = True
        while run:
            run = proto.process()  # includes auto-timeout after 30 seconds of inactivity
        proto.close()

class Server():
    def __init__(self, host, port=0):
        self.serv = socket.socket()
        self.serv.bind((host, port))
        self.serv.listen(20)
        self.cache = {}
        self.sess = requests.Session()
        self.running = False
        self.lock = threading.Lock()

    def mainloop(self):
        self.running = True
        while self.running:
            rl, wl, xl = select.select([self.serv], [], [], 0.1)
            if rl:
                client, addr = self.serv.accept()
                print("Got connection from %r"%str(addr))
                HandleIt(client, self.cache, self.sess, self.lock)

    def stop(self):
        self.running = False

def main():
    print("Initializing server at (%r, %r)"%("localhost", 9876))
    s = Server("localhost", 9876)
    s.mainloop()

if __name__ == "__nope__":
    try:
        main()
    except Exception:
        input()

In [32]:
try:
    s.stop()
    t.join()
    s.serv.close()
except:
    pass

In [33]:
s = Server("localhost",9876)
def omain(s):
    s.mainloop()
t = threading.Thread(target=omain, args=(s,))
t.start()

In [34]:
c = ClientProtocol("localhost", 9876)

Got connection from "('127.0.0.1', 58781)"


In [36]:
c.get("http://www.google.com").content

returning cached value for ''http://www.google.com''

b'<!doctype html><html itemscope="" itemtype="http://schema.org/WebPage" lang="en"><head><meta content="Search the world\'s information, including webpages, images, videos and more. Google has many special features to help you find exactly what you\'re looking for." name="description"><meta content="noodp" name="robots"><meta content="text/html; charset=UTF-8" http-equiv="Content-Type"><meta content="/images/branding/googleg/1x/googleg_standard_color_128dp.png" itemprop="image"><title>Google</title><script nonce="JmHULOl36Vu/FdPtYFmT/Q==">(function(){window.google={kEI:\'HH50XIufL_200PEPkrqwgA4\',kEXPI:\'0,18168,1335579,57,1958,1016,1406,698,527,731,1798,30,1228,805,18,71,867,202,25,311,140,39,26,404,2334208,317295,12208,1294,12383,4855,32691,15248,867,10761,1402,6381,3335,2,2,6801,364,1172,2147,1262,4243,224,1017,1201,260,5107,575,835,284,2,578,728,2432,58,2,1,3,1297,4323,3390,8,302,1268,773,2251,1406,1414,1923,1146,5,2,2,981,982,2485,110,3601,669,535,515,1808,1129,268,81,7,1,2,488,62